# Julia *gotchas* and how to handle them
(Inspired by http://www.stochasticlifestyle.com/7-julia-gotchas-handle/ by Chris Rackauckas.)

**One can write terribly slow code in any language, including Julia.**

Below we address common performance *gotchas* in Julia code.

# Gotcha 1: Global scope

In [1]:
a=2.0
b=3.0
function linearcombo()
  return 2a+b
end
answer = linearcombo()

@show answer;

answer = 7.0


In [4]:
a = 2

2

In [5]:
linearcombo()

7.0

The issue here is that the REPL/global scope does not guarantee that `a` and `b` are of a certain type.

In [6]:
@code_llvm linearcombo()


;  @ In[1]:4 within `linearcombo'
; Function Attrs: uwtable
define nonnull %jl_value_t addrspace(10)* @japi1_linearcombo_16422(%jl_value_t addrspace(10)*, %jl_value_t addrspace(10)**, i32) #0 {
top:
  %3 = alloca %jl_value_t addrspace(10)*, i32 3
  %gcframe = alloca %jl_value_t addrspace(10)*, i32 4
  %4 = bitcast %jl_value_t addrspace(10)** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* %4, i8 0, i32 32, i32 0, i1 false)
  %5 = alloca %jl_value_t addrspace(10)**, align 8
  store volatile %jl_value_t addrspace(10)** %1, %jl_value_t addrspace(10)*** %5, align 8
  %6 = call %jl_value_t*** inttoptr (i64 1801343456 to %jl_value_t*** ()*)() #3
  %7 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 0
  %8 = bitcast %jl_value_t addrspace(10)** %7 to i64*
  store i64 4, i64* %8
  %9 = getelementptr %jl_value_t**, %jl_value_t*** %6, i32 0
  %10 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 1
  %11 = bitcast %jl_

### How to identify and avoid this issue?

One way to identify the issue is [Traceur.jl](https://github.com/MikeInnes/Traceur.jl). It is basically a codified version of the [performance tips](https://docs.julialang.org/en/v0.6.4/manual/performance-tips/#man-performance-tips-1) in the Julia documentation.

In [7]:
using Traceur
@trace linearcombo()

┌ Info: Recompiling stale cache file C:\Users\carsten\.julia\compiled\v1.2\Traceur\QT21m.ji for Traceur [37b6cedf-1f77-55f8-9503-c64b63398394]
└ @ Base loading.jl:1240
┌ Warning: uses global variable Main.a
└ @ In[1]:-1
┌ Warning: uses global variable Main.b
└ @ In[1]:-1
┌ Warning: dynamic dispatch to 2 * Main.a
└ @ In[1]:-1
┌ Warning: dynamic dispatch to 2 * Main.a + Main.b
└ @ In[1]:-1
┌ Warning: linearcombo returns Any
└ @ In[1]:4


7.0

#### 1) Wrap code in functions.

In [8]:
function outer()
    a=2.0; b=3.0
    function linearcombo()
      return 2a+b
    end
    return linearcombo() 
end

answer = outer()

@show answer;

answer = 7.0


In [9]:
@code_llvm outer()


;  @ In[8]:2 within `outer'
; Function Attrs: uwtable
define double @julia_outer_17254() #0 {
top:
  ret double 7.000000e+00
}


This is fast.

In fact, it's not just fast, but as fast as it can be! Julia has figured out the result of the calculation at compile-time and returns **just the result (a literal)!**

(Effectively, `outer() = 7` at run-time.)

In [10]:
@trace outer()

7.0

#### 2) Declare globals as (compile-time) constants.

In [11]:
const A=2.0
const B=3.0

function Linearcombo()
  return 2A+B
end
answer = Linearcombo()

@show answer;

answer = 7.0


In [12]:
@code_llvm Linearcombo()


;  @ In[11]:5 within `Linearcombo'
; Function Attrs: uwtable
define double @julia_Linearcombo_17380() #0 {
top:
  ret double 7.000000e+00
}


In [13]:
@trace Linearcombo()

7.0

Note that the constants above are only compile-time constants, which can be modified:

In [14]:
const A=1.0

1.0

In [16]:
Linearcombo() # still returns 7, not 5

7.0

#### Take home message: Always wrap a performance critical piece of code in a self-contained function.

# Gotcha 2: Type-instabilities

What's bad for performance about the following function?

In [17]:
function g()
  x=1
  for i = 1:10
    x = x/2
  end
  return x
end

g (generic function with 1 method)

In [18]:
@code_llvm g()


;  @ In[17]:2 within `g'
; Function Attrs: uwtable
define double @julia_g_17434() #0 {
top:
;  @ In[17]:3 within `g'
  br label %L2

L2:                                               ; preds = %top, %L29
  %0 = phi double [ 4.940660e-324, %top ], [ %value_phi1, %L29 ]
  %.sroa.012.0 = phi i64 [ 1, %top ], [ %4, %L29 ]
  %tindex_phi = phi i2 [ -2, %top ], [ 1, %L29 ]
  %value_phi = phi i64 [ 1, %top ], [ %3, %L29 ]
;  @ In[17]:4 within `g'
  switch i2 %tindex_phi, label %L17 [
    i2 -2, label %L6
    i2 1, label %L19
  ]

L6:                                               ; preds = %L2
; ┌ @ int.jl:59 within `/'
; │┌ @ float.jl:271 within `float'
; ││┌ @ float.jl:256 within `Type' @ float.jl:60
     %1 = sitofp i64 %.sroa.012.0 to double
; └└└
  br label %L19

L17:                                              ; preds = %L2
  call void @jl_throw(%jl_value_t addrspace(12)* addrspacecast (%jl_value_t* inttoptr (i64 106599760 to %jl_value_t*) to %jl_value_t addrspace(12)*))
  unreachable



A more drastic example

In [20]:
rand(["Carsten", "Julia"], 2,3)

2×3 Array{String,2}:
 "Julia"  "Julia"  "Julia"  
 "Julia"  "Julia"  "Carsten"

In [21]:
f() = rand([1.0, 2, "3"])

f (generic function with 1 method)

In [22]:
@code_llvm f()


;  @ In[21]:1 within `f'
; Function Attrs: uwtable
define nonnull %jl_value_t addrspace(10)* @japi1_f_17543(%jl_value_t addrspace(10)*, %jl_value_t addrspace(10)**, i32) #0 {
top:
  %3 = alloca %jl_value_t addrspace(10)*, i32 2
  %gcframe = alloca %jl_value_t addrspace(10)*, i32 4
  %4 = bitcast %jl_value_t addrspace(10)** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* %4, i8 0, i32 32, i32 0, i1 false)
  %5 = alloca %jl_value_t addrspace(10)**, align 8
  store volatile %jl_value_t addrspace(10)** %1, %jl_value_t addrspace(10)*** %5, align 8
  %6 = alloca { i64, i64, i64, i64 }, align 8
  %7 = call %jl_value_t*** inttoptr (i64 1801343456 to %jl_value_t*** ()*)() #7
  %8 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 0
  %9 = bitcast %jl_value_t addrspace(10)** %8 to i64*
  store i64 4, i64* %9
  %10 = getelementptr %jl_value_t**, %jl_value_t*** %7, i32 0
  %11 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe

### How to find and deal with type-instabilities

#### 1) Avoid type changes

Initialize `x` as `Float64` and it's fast.

In [23]:
function h()
  x=1.0
  for i = 1:10
    x = x/2
  end
  return x
end

h (generic function with 1 method)

In [24]:
@code_llvm h()


;  @ In[23]:2 within `h'
; Function Attrs: uwtable
define double @julia_h_17557() #0 {
top:
;  @ In[23]:6 within `h'
  ret double 0x3F50000000000000
}


#### 2) Detect issues with `@code_warntype` (or `@trace`)

In [25]:
@code_warntype g()

Variables
  #self#::Core.Compiler.Const(g, false)
  x::Union{Float64, Int64}
  @_3::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Float64
1 ─       (x = 1)
│   %2  = (1:10)::Core.Compiler.Const(1:10, false)
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3::Core.Compiler.Const((1, 1), false) === nothing)::Core.Compiler.Const(false, false)
│   %5  = Base.not_int(%4)::Core.Compiler.Const(true, false)
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (x = x / 2)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return x::Float64


(On a side note: since the type can only vary between `Float64` and `Int64`, Julia can still produce reasonable code by *union splitting*. See the blog post by Tim Holy: https://julialang.org/blog/2018/08/union-splitting)

In [26]:
@code_warntype h()

Variables
  #self#::Core.Compiler.Const(h, false)
  x::Float64
  @_3::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Float64
1 ─       (x = 1.0)
│   %2  = (1:10)::Core.Compiler.Const(1:10, false)
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3::Core.Compiler.Const((1, 1), false) === nothing)::Core.Compiler.Const(false, false)
│   %5  = Base.not_int(%4)::Core.Compiler.Const(true, false)
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (x = x / 2)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return x


#### 3) The C/Fortran way: specify types (to get errors or to heal the problem by conversion)

In [27]:
function g2()
  x::Int64 = 1
  for i = 1:10
    x = x/2
  end
  return x
end

g2 (generic function with 1 method)

In [28]:
g2()

InexactError: InexactError: Int64(0.5)

In [29]:
function g3()
  x::Float64 = 1 # triggers an implicit conversion to Float64
  for i = 1:10
    x = x/2
  end
  return x
end

g3 (generic function with 1 method)

In [30]:
@code_llvm g3()


;  @ In[29]:2 within `g3'
; Function Attrs: uwtable
define double @julia_g3_17709() #0 {
top:
;  @ In[29]:6 within `g3'
  ret double 0x3F50000000000000
}


#### 4) Function barriers

In [31]:
arr = Vector{Union{Int64,Float64}}(undef, 4)
arr[1]=4
arr[2]=2.0
arr[3]=3.2
arr[4]=1
arr

4-element Array{Union{Float64, Int64},1}:
 4  
 2.0
 3.2
 1  

In [32]:
function foo(array)
  for i in eachindex(array)
    val = array[i]
    val^2
  end
end

foo (generic function with 1 method)

In [33]:
@code_warntype foo(arr)

Variables
  #self#::Core.Compiler.Const(foo, false)
  array::Array{Union{Float64, Int64},1}
  @_3::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  val::Union{Float64, Int64}

Body::Nothing
1 ─ %1  = Main.eachindex(array)::Base.OneTo{Int64}
│         (@_3 = Base.iterate(%1))
│   %3  = (@_3 === nothing)::Bool
│   %4  = Base.not_int(%3)::Bool
└──       goto #4 if not %4
2 ┄ %6  = @_3::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%6, 1))
│   %8  = Core.getfield(%6, 2)::Int64
│         (val = Base.getindex(array, i))
│   %10 = val::Union{Float64, Int64}
│   %11 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %12 = (%11)()::Core.Compiler.Const(Val{2}(), false)
│         Base.literal_pow(Main.:^, %10, %12)
│         (@_3 = Base.iterate(%1, %8))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return


In [34]:
function inner_foo(val)
  # Do algorithm X on val
  val^2
end
 
function foo2(array)
  for i in eachindex(array)
    inner_foo(array[i])
  end
end

foo2 (generic function with 1 method)

In [35]:
@code_warntype inner_foo(arr[1])

Variables
  #self#::Core.Compiler.Const(inner_foo, false)
  val::Int64

Body::Int64
1 ─ %1 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %2 = (%1)()::Core.Compiler.Const(Val{2}(), false)
│   %3 = Base.literal_pow(Main.:^, val, %2)::Int64
└──      return %3


#### Comments:

Why Allow Type-Instabilities in the first place? Convenience vs performance tradeoff.

Note that type instabilities can naturally occur (reading files, user input etc.) so not any red marker is bad/avoidable.

Note that Julia is smart and a changing type isn't *per se* an issue:

In [36]:
function g4()
  x=1
  x=1.0
  for i = 1:10
    x = x/2
  end
  return x
end

g4 (generic function with 1 method)

In [37]:
@code_llvm g4()


;  @ In[36]:2 within `g4'
; Function Attrs: uwtable
define double @julia_g4_17799() #0 {
top:
;  @ In[36]:7 within `g4'
  ret double 0x3F50000000000000
}


#### Take home message: watch out for type-instabilities in performance critical parts of your code.

# Gotcha 3: Views and copies

In [39]:
a = [3;4;5]
b = a
b[1] = 1
a

3-element Array{Int64,1}:
 1
 4
 5

In [42]:
a = rand(2,2)

2×2 Array{Float64,2}:
 0.387228  0.880526
 0.820225  0.9313  

In [43]:
b = vec(a) # Makes a view to the 2x2 matrix which is a 1-dimensional array

4-element Array{Float64,1}:
 0.3872277508883577
 0.8202252293238483
 0.880526408978775 
 0.931300457682325 

In [44]:
c = a[1:2,1] # Creates a copy (slice on rhs of assignment)

2-element Array{Float64,1}:
 0.3872277508883577
 0.8202252293238483

In [45]:
# Create a view into array a.
d = @view a[1:2,1]
e = view(a,1:2,1)
@views p = a[1:2,1]

2-element view(::Array{Float64,2}, 1:2, 1) with eltype Float64:
 0.3872277508883577
 0.8202252293238483

In [46]:
a[1:2,1] = [1;2] # Modifies a in-place (slice on lhs of assignment)

2-element Array{Int64,1}:
 1
 2

In [47]:
a = Vector{Vector{Float64}}(undef, 2)
a[1] = [1;2;3]
a[2] = [4;5;6]

b = copy(a)
b[1][1] = 10 # will alter a!

b = deepcopy(a) # "recursive copy"

2-element Array{Array{Float64,1},1}:
 [10.0, 2.0, 3.0]
 [4.0, 5.0, 6.0] 

# Gotcha 4: Temporary allocations and vectorized code

In [48]:
using BenchmarkTools

In [49]:
function f()
  x = [1;5;6]
  for i in 1:100_000
    x = x + 2*x
  end
  return x
end

f (generic function with 1 method)

In [50]:
@btime f();

  6.620 ms (200001 allocations: 21.36 MiB)


### How to handle it? -> More dots or more explicity

Great blog post by Steven G. Johnson: https://julialang.org/blog/2017/01/moredots ([related notebook](https://github.com/JuliaLang/www.julialang.org/blob/master/blog/_posts/moredots/More-Dots.ipynb))

In [51]:
function f()
    x = [1;5;6]
    for i in 1:100_000    
        for k in 1:3
            x[k] = x[k] + 2 * x[k]
        end
    end
    return x
end
@btime f();

  179.899 μs (1 allocation: 112 bytes)


In [58]:
a = [1,2,3]
b = [4,5,6]

3-element Array{Int64,1}:
 4
 5
 6

In [59]:
b .= a

3-element Array{Int64,1}:
 1
 2
 3

In [60]:
b === a

false

In [52]:
function f()
    x = [1;5;6]
    for i in 1:100_000
        x = x .+ 2 .* x
    end
    return x
end
@btime f();

  2.524 ms (100001 allocations: 10.68 MiB)


In [53]:
function f()
    x = [1;5;6]
    for i in 1:100_000
        x .= x .+ 2 .* x
    end
    return x
end
@btime f();

  283.099 μs (1 allocation: 112 bytes)


In [61]:
function f()
    x = [1;5;6]
    for i in 1:100_000
        @. x = x + 2*x
        # or @. x = x + 2*x
    end
    return x
end
@btime f();

  283.099 μs (1 allocation: 112 bytes)


### Extra trick: `@inbounds`

In [ ]:
function f()
    x = [1;5;6]
    @inbounds for i in 1:100_000    
        for k in 1:length(x)
            x[k] = x[k] + 2*x[k]
        end
    end
    return x
end
@btime f();

# Gotcha 5: Abstract fields

In [62]:
using BenchmarkTools

In [ ]:
struct A
    x::Any
    y::Any
end

In [63]:
struct MyType
    x::AbstractFloat
    y::AbstractString
end

f(a::MyType) = a.x^2 + sqrt(a.x)

f (generic function with 2 methods)

In [64]:
a = MyType(3.0, "test")

@btime f($a);

  48.382 ns (3 allocations: 48 bytes)


In [65]:
struct MyTypeConcrete
    x::Float64
    y::String
end

f(b::MyTypeConcrete) = b.x^2 + sqrt(b.x)

f (generic function with 3 methods)

In [66]:
b = MyTypeConcrete(3.0, "test")

@btime f($b);

  1.499 ns (0 allocations: 0 bytes)


Note that the latter implementation is **more than 30x faster**!

### How to handle it?

But what if I want to accept any kind of `AbstractFloat` and `AbstractString` in my type?

Use type parameters!

In [67]:
struct MyTypeParametric{A<:AbstractFloat, B<:AbstractString}
    x::A
    y::B
end

f(c::MyTypeParametric) = c.x^2 + sqrt(c.x)

f (generic function with 4 methods)

In [68]:
c = MyTypeParametric(3.0, "test")

MyTypeParametric{Float64,String}(3.0, "test")

From the type alone the compiler knows what the structure contains and can produce optimal code:

In [72]:
@btime f($c);

  1.299 ns (0 allocations: 0 bytes)


In [73]:
c = MyTypeParametric(Float32(3.0), SubString("test"))

MyTypeParametric{Float32,SubString{String}}(3.0f0, "test")

In [74]:
@btime f($c);

  1.299 ns (0 allocations: 0 bytes)


In [75]:
a = 4

4

In [76]:
"The greatest number in the world is $a"

"The greatest number in the world is 4"

# Gotcha 6: Writing to global scope

In [ ]:
# Try this in the Julia REPL
a = 0
for i in 1:10
    a += i
end

(For more information, see the "official" discussion here: https://github.com/JuliaLang/julia/issues/28789)

#### Take home message: again, just wrap things into functions.

# Gotcha 7: Column major order

In [77]:
M = rand(1000,1000);

function fcol(M)
    for col in 1:size(M, 2)
        for row in 1:size(M, 1)
            M[row, col] = 42
        end
    end
    nothing
end

function frow(M)
    for row in 1:size(M, 1)
        for col in 1:size(M, 2)
            M[row, col] = 42
        end
    end
    nothing
end

frow (generic function with 1 method)

In [ ]:
M .= 42

In [78]:
@btime fcol($M)

  372.700 μs (0 allocations: 0 bytes)


In [79]:
@btime frow($M)

  1.447 ms (0 allocations: 0 bytes)


#### Take home message: fastest varying index goes first!

# Gotcha 8: Lazy operations

Let's say we want to calculate `X = M + (M' + 2*I)`.

In [80]:
using LinearAlgebra

In [81]:
a = rand(2,2)

2×2 Array{Float64,2}:
 0.895057  0.901672
 0.574545  0.453461

In [88]:
b = Matrix(a')

2×2 Array{Float64,2}:
 5.0       0.574545
 0.901672  0.453461

In [85]:
b[1] = 5

5

In [86]:
a

2×2 Array{Float64,2}:
 5.0       0.901672
 0.574545  0.453461

In [83]:
M = [1 2; 3 4]
M + (M' + 2*I)

2×2 Array{Int64,2}:
 4   5
 5  10

Now let's assume that, for some reason, we want to implement it more explicitly. Something along the lines of

In [ ]:
function calc(M)
    X = M'
    X[1,1] += 2
    X[2,2] += 2
    M + X
end

Let's check for correctness.

In [ ]:
calc([1 2; 3 4]) == M + (M' + 2*I)

Somehow it's not correct!

### How to solve this?

The "issue" is that `M'` makes a lazy adjoint of `M`. It is just another way of looking at the same piece of memory. Hence, when we do `X[1,1] += 1` we are actually changing `M`, leading to a wrong result. We can heal this by enforcing a `copy`:

In [ ]:
function calc_corrected(M)
    X = copy(M')
    X[diagind(X)] .+= 2
    M + X
end

In [ ]:
calc_corrected([1 2; 3 4]) == M + (M' + 2*I)

This isn't really an issue. In fact, this lazyness (+ allocation free identity matrix) is precisley the reason why the straightforward solution is fast!

In [ ]:
function calc_straightforward(A)
    A + (A' + 2*I)
end

@btime calc($[1 2; 3 4]);
@btime calc_corrected($[1 2; 3 4]);
@btime calc_straightforward($[1 2; 3 4]);

### Extra tip: Comprehensions and generators

In [ ]:
[k for k in 1:10]

The construct is known as a [comprehension](https://docs.julialang.org/en/v1/manual/arrays/#Comprehensions-1).

In [ ]:
sum([k for k in 1:10])

To avoid the temporary array that the comprehension creates, we can also write the comprehension withouth square brackets. This creates a so-called [generator expression](https://docs.julialang.org/en/v1/manual/arrays/#Generator-Expressions-1).

In [ ]:
sum(k for k in 1:10)

In [ ]:
gen = (k for k in 1:10)

In [ ]:
collect(gen)

In [ ]:
using BenchmarkTools

@btime sum([k for k in 1:10]);
@btime sum(k for k in 1:10);

# Core messages of this Notebook

* Gotcha 1: **Wrap code in self-contained functions** in performance critical applications, i.e. avoid global scope.
* Gotcha 2: Write **type-stable code** (check with `@code_warntype`).
* Gotcha 3: Use **views** instead of copies to avoid unnecessary allocations.
* Gotcha 4: Use **broadcasting (more dots)** to avoid temporary allocations in vectorized code (or write out loops).
* Gotcha 5: **Types should always have concrete fields.** If you don't know them in advance, use type parameters.


* Gotcha 6: Be aware of the **scoping rules** in non-Jupyter-notebook environments.
* Gotcha 7: Be aware of **column major order** when looping over arrays.
* Gotcha 8: Be aware of **lazy operations** like, for example, transpose.